## Name: Naman Trisoliya

## ID: 12141150


# Loading libraries


In [1]:
import random
import torch
import numpy as np
import torchvision
import torch.nn as nn
from tqdm import tqdm
import torch.nn.functional as F
import torch.optim as optim
from torchsummary import summary
from torch.optim.lr_scheduler import MultiStepLR
from torch.utils.data import DataLoader, Subset
from torchvision import datasets, transforms
import matplotlib.pyplot as plt

In [2]:
device=torch.device('cuda' if torch.cuda.is_available() else 'cpu')
device

device(type='cpu')

# Loading Dataset and preprocessing


In [3]:
transform=transforms.Compose([transforms.ToTensor(),
                    transforms.Normalize((0.5),(0.5))])

In [4]:
data_train=datasets.MNIST(root='./data',train=True,download=True,transform=transform)
data_test=datasets.MNIST(root='./data',train=False,download=True,transform=transform)

In [5]:
print(len(data_train),len(data_test))

60000 10000


## Dataset setup for experiments


The experiments were performed with 200 random points taken from each class during training and 10000 points in test set.


In [6]:
batch_size=50

In [7]:
def get_subset_dataloader(dataset, random_seed, num_samples_per_class,batch_size):
    torch.manual_seed(random_seed)

    class_indices = {}
    for i in range(len(dataset.classes)):
        class_indices[i] = []

    for idx, (_, label) in enumerate(dataset):
        class_indices[label].append(idx)

    selected_indices = []
    for _, indices in class_indices.items():
        selected_indices.extend(random.sample(indices, num_samples_per_class))

    subset_dataset = Subset(dataset, selected_indices)
    # print("Subset created with total lenght:",len(subset_dataset))

    dataloader = DataLoader(subset_dataset, batch_size=batch_size, shuffle=True)

    return dataloader

In [8]:
test_loader=torch.utils.data.DataLoader(data_test,batch_size=batch_size,shuffle=False)

In [9]:
classes = ('Zero','One', 'Two', 'Three', 'Four','Five', 'Six', 'Seven', 'Eight', 'Nine')

# Model Building


LeNet-5 Architecture was used in the experiments with maxpooling, ReLU activations and dropout after the dense hidden layer. The total parameter count of the used baseline model is 61706.


In [10]:
class BaseModel(nn.Module):
    def __init__(self,include_dropout):
        super(BaseModel, self).__init__()
        self.include_dropout=include_dropout

        self.conv1 = nn.Conv2d(in_channels=1, out_channels=6, kernel_size=5,stride=1,padding=2)
        self.conv2 = nn.Conv2d(in_channels=6, out_channels=16, kernel_size=5,stride=1)


        self.maxpool = nn.MaxPool2d(kernel_size=2, stride=2)

        self.fc1 = nn.Linear(16 * 5 * 5, 120)
        self.fc2 = nn.Linear(120, 84)
        self.fc3 = nn.Linear(84, 10)

        self.dropout = nn.Dropout(p=0.5)

        self.flatten=nn.Flatten()

    def forward(self, x):
        x = self.maxpool(F.relu(self.conv1(x)))

        x = self.maxpool(F.relu(self.conv2(x)))

        x = self.flatten(x)

        x = F.relu(self.fc1(x))
        if self.include_dropout:
            x = self.dropout(x)

        x = F.relu(self.fc2(x))
        if self.include_dropout:
            x = self.dropout(x)

        x = self.fc3(x)
        return x

model= BaseModel(include_dropout=False)
model.to(device)
summary(model,(1,28,28))

----------------------------------------------------------------
        Layer (type)               Output Shape         Param #
            Conv2d-1            [-1, 6, 28, 28]             156
         MaxPool2d-2            [-1, 6, 14, 14]               0
            Conv2d-3           [-1, 16, 10, 10]           2,416
         MaxPool2d-4             [-1, 16, 5, 5]               0
           Flatten-5                  [-1, 400]               0
            Linear-6                  [-1, 120]          48,120
            Linear-7                   [-1, 84]          10,164
            Linear-8                   [-1, 10]             850
Total params: 61,706
Trainable params: 61,706
Non-trainable params: 0
----------------------------------------------------------------
Input size (MB): 0.00
Forward/backward pass size (MB): 0.06
Params size (MB): 0.24
Estimated Total Size (MB): 0.30
----------------------------------------------------------------


In [11]:
class BatchNormModel(nn.Module):
    def __init__(self):
        super(BatchNormModel, self).__init__()

        self.conv1 = nn.Conv2d(in_channels=1, out_channels=6, kernel_size=5,stride=1,padding=2)
        self.conv2 = nn.Conv2d(in_channels=6, out_channels=16, kernel_size=5,stride=1)


        self.maxpool = nn.MaxPool2d(kernel_size=2, stride=2)
        self.norm1 = nn.BatchNorm2d(6)
        self.norm2 = nn.BatchNorm2d(16)
        self.norm3 = nn.BatchNorm1d(120)
        self.norm4 = nn.BatchNorm1d(84)


        self.fc1 = nn.Linear(16 * 5 * 5, 120)
        self.fc2 = nn.Linear(120, 84)
        self.fc3 = nn.Linear(84, 10)

        self.dropout = nn.Dropout(p=0.5)

        self.flatten=nn.Flatten()

    def forward(self, x):
        x = self.maxpool(self.norm1(F.relu(self.conv1(x))))

        x = self.maxpool(self.norm2(F.relu(self.conv2(x))))

        x = self.flatten(x)

        x = self.norm3(F.relu(self.fc1(x)))

        x = self.norm4(F.relu(self.fc2(x)))

        x = self.fc3(x)
        return x

model= BatchNormModel()
model.to(device)
summary(model,(1,28,28))

----------------------------------------------------------------
        Layer (type)               Output Shape         Param #
            Conv2d-1            [-1, 6, 28, 28]             156
       BatchNorm2d-2            [-1, 6, 28, 28]              12
         MaxPool2d-3            [-1, 6, 14, 14]               0
            Conv2d-4           [-1, 16, 10, 10]           2,416
       BatchNorm2d-5           [-1, 16, 10, 10]              32
         MaxPool2d-6             [-1, 16, 5, 5]               0
           Flatten-7                  [-1, 400]               0
            Linear-8                  [-1, 120]          48,120
       BatchNorm1d-9                  [-1, 120]             240
           Linear-10                   [-1, 84]          10,164
      BatchNorm1d-11                   [-1, 84]             168
           Linear-12                   [-1, 10]             850
Total params: 62,158
Trainable params: 62,158
Non-trainable params: 0
---------------------------------

In [12]:
class DropoutModel(nn.Module):
    def __init__(self):
        super(DropoutModel, self).__init__()

        self.conv1 = nn.Conv2d(in_channels=1, out_channels=6, kernel_size=5,stride=1,padding=2)
        self.conv2 = nn.Conv2d(in_channels=6, out_channels=16, kernel_size=5,stride=1)


        self.maxpool = nn.MaxPool2d(kernel_size=2, stride=2)

        self.fc1 = nn.Linear(16 * 5 * 5, 120)
        self.fc2 = nn.Linear(120, 84)
        self.fc3 = nn.Linear(84, 10)

        self.dropout = nn.Dropout(p=0.5)

        self.flatten=nn.Flatten()

    def forward(self, x):
        x = self.maxpool(F.relu(self.conv1(x)))
        x = self.dropout(x)

        x = self.maxpool(F.relu(self.conv2(x)))
        x = self.dropout(x)

        x = self.flatten(x)

        x = F.relu(self.fc1(x))
        x = self.dropout(x)

        x = F.relu(self.fc2(x))
        x = self.dropout(x)

        x = self.fc3(x)
        return x

model= DropoutModel()
model.to(device)
summary(model,(1,28,28))

----------------------------------------------------------------
        Layer (type)               Output Shape         Param #
            Conv2d-1            [-1, 6, 28, 28]             156
         MaxPool2d-2            [-1, 6, 14, 14]               0
           Dropout-3            [-1, 6, 14, 14]               0
            Conv2d-4           [-1, 16, 10, 10]           2,416
         MaxPool2d-5             [-1, 16, 5, 5]               0
           Dropout-6             [-1, 16, 5, 5]               0
           Flatten-7                  [-1, 400]               0
            Linear-8                  [-1, 120]          48,120
           Dropout-9                  [-1, 120]               0
           Linear-10                   [-1, 84]          10,164
          Dropout-11                   [-1, 84]               0
           Linear-12                   [-1, 10]             850
Total params: 61,706
Trainable params: 61,706
Non-trainable params: 0
---------------------------------

# Metrics Setup


In [13]:
def calculate_accuracy(output,labels):
    _,predicted=torch.max(output.data, 1)

    total = labels.size(0)
    correct = (predicted.to(device) == labels).sum().item()

    return correct,total

# Regularizers code setup


In [14]:
def double_backprop_loss(model, criterion, inputs, targets, lambda_double_backprop):
    outputs = model(inputs)
    loss = criterion(outputs, targets)

    # Compute gradient of the original loss with respect to model parameters
    model.zero_grad()
    loss.backward(retain_graph=True)
    grad_loss = torch.cat([param.grad.flatten() for param in model.parameters()])

    # Compute squared L2 norm of the gradient
    double_backprop_term = lambda_double_backprop * torch.norm(grad_loss, p=2)**2

    # Total loss including Double Backpropagation regularization
    total_loss = loss + double_backprop_term

    return total_loss

In [15]:
def jac_reg_loss(model, criterion, inputs, targets, lambda_jacreg):
    outputs = model(inputs)
    loss = criterion(outputs, targets)

    # Compute Jacobian of softmax output probabilities w.r.t. input
    inputs.requires_grad_(True)
    outputs = model(inputs)
    outputs=F.softmax(outputs,dim=1)
    grad_outputs = torch.ones_like(outputs)
    grad_softmax = torch.autograd.grad(outputs, inputs, grad_outputs=grad_outputs, retain_graph=True)[0]

    # Compute squared Frobenius norm of Jacobian
    jac_norm_squared = torch.norm(grad_softmax, p='fro')**2

    # Jacobian Regularization term
    jacreg_term = lambda_jacreg * jac_norm_squared

    # Total loss including JacReg
    total_loss = loss + jacreg_term

    return total_loss

In [16]:
def frobreg_loss(model, criterion, inputs, targets, lambda_frobreg):
    outputs = model(inputs)
    loss = criterion(outputs, targets)

    # Compute Jacobian of logits w.r.t. input
    inputs.requires_grad_(True)
    outputs = model(inputs)
    grad_outputs = torch.ones_like(outputs)
    grad_logits = torch.autograd.grad(outputs, inputs, grad_outputs=grad_outputs, retain_graph=True)[0]

    # Compute squared Frobenius norm of Jacobian
    frob_norm_squared = torch.norm(grad_logits, p='fro')**2

    # Frobenius Regularization term
    frobreg_term = lambda_frobreg * frob_norm_squared

    # Total loss including FrobReg
    total_loss = loss + frobreg_term

    return total_loss

In [17]:
def spectral_reg_loss(model, criterion, inputs, targets, lambda_spectreg, k_dimensions):
    outputs = model(inputs)
    loss = criterion(outputs, targets)

    # Compute Jacobian of logits w.r.t. input
    inputs.requires_grad_(True)
    outputs = model(inputs)
    grad_outputs = torch.ones_like(outputs)
    grad_logits = torch.autograd.grad(outputs, inputs, grad_outputs=grad_outputs, retain_graph=True)[0]

    # Apply random projection
    random_proj = torch.randn(grad_logits.shape[2], k_dimensions).to(device)
    # print(grad_logits.shape,random_proj.shape)
    projected_grad_logits = torch.matmul(grad_logits, random_proj)

    # Compute squared L2 norm of the result
    spectral_reg_term = lambda_spectreg * torch.norm(projected_grad_logits, p=2)**2

    # Total loss including SpectReg
    total_loss = loss + spectral_reg_term

    return total_loss

In [18]:
def conf_penalty_loss(model,criterion,inputs,targets,alpha=0.1):
    outputs = model(inputs)
    loss = criterion(outputs, targets)
    # Calculate probabilities from logits using softmax
    probs = torch.softmax(outputs, dim=1)

    # Calculate entropy
    entropy = torch.sum(-probs * torch.log(probs + 1e-10), dim=1).mean()

    # Apply confidence penalty regularization
    penalty = alpha * entropy

    total_loss = loss + penalty

    return total_loss

In [19]:
def Evaluate(test_loader,model):
    with torch.no_grad():
        val_correct=0
        val_total=0
        for i, data in enumerate(test_loader):
            inputs,label=data

            inputs=inputs.to(device)
            label=label.to(device)

            output=model(inputs)
            correct,total=calculate_accuracy(output,label)
            val_correct+=correct
            val_total+=total
    # print(val_correct,val_total)
    return round(100*(val_correct/val_total),2)

# Experiment C2: GRADIENT REGULARIZATION COMPARED WITH DROPOUT AND BATCH NORMALIZATION


In [ ]:
batch_size=50
num_samples_per_class=200
seeds=[10,20,30,40,50,60,70,80,90,100]

In [ ]:
def train(model,train_loader,test_loader,num_samples_per_class,regularizer,reg_parameter):
    criterion=nn.CrossEntropyLoss()
    opt=torch.optim.Adam(model.parameters(),lr=0.01,betas=(0.9,0.999),weight_decay=0.0005)

    num_epochs= int(10000/((num_samples_per_class*10)/batch_size))
    # num_epochs=1
    milestones = [int(0.5 * num_epochs), int(0.75 * num_epochs)]
    scheduler = MultiStepLR(opt, milestones=milestones, gamma=0.1)

    for epoch in tqdm(range(num_epochs),"Trained"):
        for i, data in enumerate(train_loader):
            inputs,label=data
            inputs=inputs.to(device)
            label=label.to(device)
            opt.zero_grad()

            if regularizer=="SpectReg":
                loss = spectral_reg_loss(model, criterion, inputs, label, reg_parameter, 10)
            elif regularizer=="NoGR":
                output=model(inputs)
                loss = criterion(output,label)
            elif regularizer=="DoubleBack":
                loss = double_backprop_loss(model, criterion, inputs, label,reg_parameter)

            loss.backward()
            opt.step()

        scheduler.step()

    return Evaluate(test_loader,model)

In [ ]:
models=["Baseline","Batchnorm","Dropout"]
regularizers=["NoGR","DoubleBack","SpectReg"]

model_params={"Baseline":{"NoGR":0,"DoubleBack":50,"SpectReg":0.01},
              "Batchnorm":{"NoGR":0,"DoubleBack":0.001,"SpectReg":0.001},
              "Dropout":{"NoGR":0,"DoubleBack":50,"SpectReg":0.01}}

accuracy_dict={}
for i in models:
    k={}
    for j in regularizers:
        k[j]=[]
    accuracy_dict[i]=k
print(accuracy_dict)

{'Baseline': {'NoGR': [], 'DoubleBack': [], 'SpectReg': []}, 'Batchnorm': {'NoGR': [], 'DoubleBack': [], 'SpectReg': []}, 'Dropout': {'NoGR': [], 'DoubleBack': [], 'SpectReg': []}}


In [ ]:
for model_name in models:
    for reg in regularizers:
            reg_parameter=model_params[model_name][reg]
            print(f"Model Name:{model_name}, Regularizer: {reg}, Parameter: {reg_parameter}")
            for i in range(10):
                print(f"Iteration:{i} seed:{seeds[i]}")

                if model_name=="Baseline":
                    model=BaseModel(include_dropout=False).to(device)
                elif model_name=="Batchnorm":
                    model=BatchNormModel().to(device)
                elif model_name== "Dropout":
                    model=DropoutModel().to(device)

                train_loader=get_subset_dataloader(data_train,seeds[i],num_samples_per_class,batch_size)
                accuracy=train(model,train_loader,test_loader,num_samples_per_class,reg,reg_parameter)
                accuracy_dict[model_name][reg].append(accuracy)
                print("Accuracy:",accuracy)

            accuracy_list=accuracy_dict[model_name][reg]
            print(f"Mean Accuracy:{np.mean(accuracy_list)}, STD: {np.std(accuracy_list)}")
            print("")

Model Name:Baseline, Regularizer: NoGR, Parameter: 0
Iteration:0 seed:10


Trained: 100%|██████████| 250/250 [02:10<00:00,  1.91it/s]


Accuracy: 96.09
Iteration:1 seed:20


Trained: 100%|██████████| 250/250 [02:12<00:00,  1.89it/s]


Accuracy: 97.23
Iteration:2 seed:30


...

Trained: 100%|██████████| 250/250 [02:11<00:00,  1.91it/s]


Accuracy: 91.76
Iteration:9 seed:100


Trained: 100%|██████████| 250/250 [02:09<00:00,  1.93it/s]


Accuracy: 91.68
Mean Accuracy:91.902, STD: 0.2833301960610621

Model Name:Dropout, Regularizer: DoubleBack, Parameter: 50
Iteration:0 seed:10


Trained:  59%|█████▉    | 147/250 [01:23<00:57,  1.80it/s]

In [ ]:
models=["Dropout"]
regularizers=["DoubleBack","SpectReg"]

model_params={"Baseline":{"NoGR":0,"DoubleBack":50,"SpectReg":0.01},
              "Batchnorm":{"NoGR":0,"DoubleBack":0.001,"SpectReg":0.001},
              "Dropout":{"NoGR":0,"DoubleBack":50,"SpectReg":0.01}}

accuracy_dict={}
for i in models:
    k={}
    for j in regularizers:
        k[j]=[]
    accuracy_dict[i]=k
print(accuracy_dict)

{'Dropout': {'DoubleBack': [], 'SpectReg': []}}


In [ ]:
for model_name in models:
    for reg in regularizers:
            reg_parameter=model_params[model_name][reg]
            print(f"Model Name:{model_name}, Regularizer: {reg}, Parameter: {reg_parameter}")
            for i in range(10):
                print(f"Iteration:{i} seed:{seeds[i]}")

                if model_name=="Baseline":
                    model=BaseModel(include_dropout=False).to(device)
                elif model_name=="Batchnorm":
                    model=BatchNormModel().to(device)
                elif model_name== "Dropout":
                    model=DropoutModel().to(device)

                train_loader=get_subset_dataloader(data_train,seeds[i],num_samples_per_class,batch_size)
                accuracy=train(model,train_loader,test_loader,num_samples_per_class,reg,reg_parameter)
                accuracy_dict[model_name][reg].append(accuracy)
                print("Accuracy:",accuracy)

            accuracy_list=accuracy_dict[model_name][reg]
            print(f"Mean Accuracy:{np.mean(accuracy_list)}, STD: {np.std(accuracy_list)}")
            print("")

Model Name:Dropout, Regularizer: DoubleBack, Parameter: 50
Iteration:0 seed:10


Trained: 100%|██████████| 250/250 [02:18<00:00,  1.81it/s]


Accuracy: 91.57
Iteration:1 seed:20


Trained: 100%|██████████| 250/250 [02:18<00:00,  1.81it/s]


Accuracy: 91.91
Iteration:2 seed:30


...

Accuracy: 90.78
Iteration:8 seed:90


Trained: 100%|██████████| 250/250 [02:21<00:00,  1.76it/s]


Accuracy: 91.77
Iteration:9 seed:100


Trained: 100%|██████████| 250/250 [02:21<00:00,  1.77it/s]


Accuracy: 91.35
Mean Accuracy:91.651, STD: 0.6438858594502621



# Experiment C4: THE EFFECT OF TRAINING SET SIZE


In [20]:
batch_size=50
seeds=[10,20,30,40,50,60,70,80,90,100]

In [21]:
def train(model,train_loader,test_loader,num_samples_per_class,regularizer,reg_parameter):
    criterion=nn.CrossEntropyLoss()
    opt=torch.optim.Adam(model.parameters(),lr=0.001,betas=(0.9,0.999),weight_decay=0.0005)
    
    num_epochs= int(10000/((num_samples_per_class*10)/batch_size))
    # num_epochs=1
    milestones = [int(0.5 * num_epochs), int(0.75 * num_epochs)]  
    scheduler = MultiStepLR(opt, milestones=milestones, gamma=0.1)
    
    for epoch in tqdm(range(num_epochs),"Trained"):
        for i, data in enumerate(train_loader):
            inputs,label=data
            inputs=inputs.to(device)
            label=label.to(device)
            opt.zero_grad()

            if regularizer=="SpectReg":
                loss = spectral_reg_loss(model, criterion, inputs, label, reg_parameter, 10)
            elif regularizer=="JacReg":
                loss = jac_reg_loss(model, criterion, inputs, label, reg_parameter)
            elif regularizer=="DoubleBack":
                loss = double_backprop_loss(model, criterion, inputs, label,reg_parameter)
            elif regularizer=='ConfPenalty':
                loss = conf_penalty_loss(model, criterion, inputs, label, reg_parameter)

            loss.backward()
            opt.step()
    
        scheduler.step()
         
    return Evaluate(test_loader,model)

In [22]:
train_size=[500,1000,2000,3000,4000,5000,10000,15000,20000]
doubleback_par=[50,50,50,20,20,20,5,2,2]
spectreg_par=[0.03,0.03,0.03,0.03,0.03,0.003,0.01,0.01,0.001]
jacreg_par=[0.3,0.03,1,1,1,1,1,1,0.3]
cp_par=[0.01,0.01,0.01,0.01,0.01,0.1,0.1,0.01,0.03]

regularizers=["DoubleBack","SpectReg","JacReg","ConfPenalty"]
params={"DoubleBack":doubleback_par,
        "SpectReg":spectreg_par,
        "JacReg":jacreg_par,
        "ConfPenalty":cp_par}

accuracy_dict={}
for i in train_size:
    k={}
    for j in regularizers:
        k[j]=[]
    accuracy_dict[i]=k


In [23]:
for index,size in enumerate(train_size):
    num_samples_per_class=size//10
    for reg in regularizers:
        reg_parameter=params[reg][index]
        print(f"Train Size: {size}, Regularizer: {reg}, Parameter: {reg_parameter}")
        for i in range(10):
            print(f"Iteration:{i} seed:{seeds[i]}")
            model=BaseModel(include_dropout=True).to(device)
            train_loader=get_subset_dataloader(data_train,seeds[i],num_samples_per_class,batch_size)
            accuracy=train(model,train_loader,test_loader,num_samples_per_class,reg,reg_parameter)
            accuracy_dict[size][reg].append(accuracy)
            print("Accuracy:",accuracy)
        accuracy_list=accuracy_dict[size][reg]
        print(f"Mean Accuracy:{np.mean(accuracy_list)}, STD: {np.std(accuracy_list)}")
        print("")

Train Size: 500, Regularizer: DoubleBack, Parameter: 50
Iteration:0 seed:10


Trained: 100%|██████████| 1000/1000 [03:04<00:00,  5.41it/s]


Accuracy: 95.28
Iteration:1 seed:20


Trained: 100%|██████████| 1000/1000 [03:28<00:00,  4.79it/s]


Accuracy: 94.44
Iteration:2 seed:30


...

Accuracy: 96.4
Mean Accuracy:96.602, STD: 0.1409822683886141

Train Size: 3000, Regularizer: DoubleBack, Parameter: 20
Iteration:0 seed:10


Trained: 100%|██████████| 166/166 [03:28<00:00,  1.26s/it]


Accuracy: 96.94
Iteration:1 seed:20


Trained:  32%|███▏      | 53/166 [01:04<02:18,  1.22s/it]


KeyboardInterrupt: 

In [ ]:
train_size=[3000,4000]
doubleback_par=[20,20]
spectreg_par=[0.03,0.03]
jacreg_par=[1,1]
cp_par=[0.01,0.01]

regularizers=["DoubleBack","SpectReg","JacReg","ConfPenalty"]
params={"DoubleBack":doubleback_par,
        "SpectReg":spectreg_par,
        "JacReg":jacreg_par,
        "ConfPenalty":cp_par}

accuracy_dict={}
for i in train_size:
    k={}
    for j in regularizers:
        k[j]=[]
    accuracy_dict[i]=k


In [ ]:
for index,size in enumerate(train_size):
    num_samples_per_class=size//10
    for reg in regularizers:
        reg_parameter=params[reg][index]
        print(f"Train Size: {size}, Regularizer: {reg}, Parameter: {reg_parameter}")
        for i in range(10):
            print(f"Iteration:{i} seed:{seeds[i]}")
            model=BaseModel(include_dropout=True).to(device)
            train_loader=get_subset_dataloader(data_train,seeds[i],num_samples_per_class,batch_size)
            accuracy=train(model,train_loader,test_loader,num_samples_per_class,reg,reg_parameter)
            accuracy_dict[size][reg].append(accuracy)
            print("Accuracy:",accuracy)
        accuracy_list=accuracy_dict[size][reg]
        print(f"Mean Accuracy:{np.mean(accuracy_list)}, STD: {np.std(accuracy_list)}")
        print("")

Train Size: 3000, Regularizer: DoubleBack, Parameter: 20
Iteration:0 seed:10


Trained: 100%|██████████| 166/166 [02:18<00:00,  1.20it/s]


Accuracy: 96.93
Iteration:1 seed:20


Trained: 100%|██████████| 166/166 [02:20<00:00,  1.18it/s]


Accuracy: 97.16
Iteration:2 seed:30


...

Accuracy: 96.99
Iteration:8 seed:90


Trained: 100%|██████████| 125/125 [02:06<00:00,  1.01s/it]


Accuracy: 97.25
Iteration:9 seed:100


Trained: 100%|██████████| 125/125 [02:05<00:00,  1.00s/it]


Accuracy: 97.9
Mean Accuracy:97.33399999999999, STD: 0.30381573362813147



In [ ]:
train_size=[5000,10000]
doubleback_par=[20,5]
spectreg_par=[0.003,0.01]
jacreg_par=[1,1]
cp_par=[0.1,0.1]

regularizers=["DoubleBack","SpectReg","JacReg","ConfPenalty"]
params={"DoubleBack":doubleback_par,
        "SpectReg":spectreg_par,
        "JacReg":jacreg_par,
        "ConfPenalty":cp_par}

accuracy_dict={}
for i in train_size:
    k={}
    for j in regularizers:
        k[j]=[]
    accuracy_dict[i]=k


In [ ]:
for index,size in enumerate(train_size):
    num_samples_per_class=size//10
    for reg in regularizers:
        reg_parameter=params[reg][index]
        print(f"Train Size: {size}, Regularizer: {reg}, Parameter: {reg_parameter}")
        for i in range(10):
            print(f"Iteration:{i} seed:{seeds[i]}")
            model=BaseModel(include_dropout=True).to(device)
            train_loader=get_subset_dataloader(data_train,seeds[i],num_samples_per_class,batch_size)
            accuracy=train(model,train_loader,test_loader,num_samples_per_class,reg,reg_parameter)
            accuracy_dict[size][reg].append(accuracy)
            print("Accuracy:",accuracy)
        accuracy_list=accuracy_dict[size][reg]
        print(f"Mean Accuracy:{np.mean(accuracy_list)}, STD: {np.std(accuracy_list)}")
        print("")

Train Size: 5000, Regularizer: DoubleBack, Parameter: 20
Iteration:0 seed:10


Trained: 100%|██████████| 100/100 [02:18<00:00,  1.38s/it]


Accuracy: 97.29
Iteration:1 seed:20


Trained: 100%|██████████| 100/100 [02:14<00:00,  1.35s/it]


Accuracy: 97.4
Iteration:2 seed:30


...

Accuracy: 97.66
Iteration:8 seed:90


Trained: 100%|██████████| 50/50 [02:02<00:00,  2.45s/it]


Accuracy: 98.06
Iteration:9 seed:100


Trained: 100%|██████████| 50/50 [02:02<00:00,  2.44s/it]


Accuracy: 97.69
Mean Accuracy:97.95899999999999, STD: 0.17166537216340585



In [ ]:
train_size=[15000,20000]
doubleback_par=[2,2]
spectreg_par=[0.01,0.001]
jacreg_par=[1,0.3]
cp_par=[0.01,0.03]

regularizers=["DoubleBack","SpectReg","JacReg","ConfPenalty"]
params={"DoubleBack":doubleback_par,
        "SpectReg":spectreg_par,
        "JacReg":jacreg_par,
        "ConfPenalty":cp_par}

accuracy_dict={}
for i in train_size:
    k={}
    for j in regularizers:
        k[j]=[]
    accuracy_dict[i]=k


In [ ]:
for index,size in enumerate(train_size):
    num_samples_per_class=size//10
    for reg in regularizers:
        reg_parameter=params[reg][index]
        print(f"Train Size: {size}, Regularizer: {reg}, Parameter: {reg_parameter}")
        for i in range(10):
            print(f"Iteration:{i} seed:{seeds[i]}")
            model=BaseModel(include_dropout=True).to(device)
            train_loader=get_subset_dataloader(data_train,seeds[i],num_samples_per_class,batch_size)
            accuracy=train(model,train_loader,test_loader,num_samples_per_class,reg,reg_parameter)
            accuracy_dict[size][reg].append(accuracy)
            print("Accuracy:",accuracy)
        accuracy_list=accuracy_dict[size][reg]
        print(f"Mean Accuracy:{np.mean(accuracy_list)}, STD: {np.std(accuracy_list)}")
        print("")

Train Size: 15000, Regularizer: DoubleBack, Parameter: 2
Iteration:0 seed:10


Trained: 100%|██████████| 33/33 [02:19<00:00,  4.24s/it]


Accuracy: 98.12
Iteration:1 seed:20


Trained: 100%|██████████| 33/33 [02:15<00:00,  4.11s/it]


Accuracy: 98.43
Iteration:2 seed:30


...

Accuracy: 97.98
Iteration:8 seed:90


Trained: 100%|██████████| 25/25 [02:04<00:00,  4.99s/it]


Accuracy: 98.4
Iteration:9 seed:100


Trained: 100%|██████████| 25/25 [02:03<00:00,  4.96s/it]


Accuracy: 98.35
Mean Accuracy:98.264, STD: 0.15047923444781133



# Experiment C6: APPROXIMATING THE FROBENIUS NORM OF THE JACOBIAN DOES NOT DECREASE ACCURACY


In [20]:
batch_size=50
num_samples_per_class=200
seeds=[10,20,30,40,50,60,70,80,90,100]

In [21]:
def train(model,train_loader,test_loader,num_samples_per_class,regularizer):
    criterion=nn.CrossEntropyLoss()
    opt=torch.optim.Adam(model.parameters(),lr=0.001,betas=(0.9,0.999),weight_decay=0.0005)
    
    num_epochs= int(10000/((num_samples_per_class*10)/batch_size))
    # num_epochs=10
    milestones = [int(0.5 * num_epochs), int(0.75 * num_epochs)]  
    scheduler = MultiStepLR(opt, milestones=milestones, gamma=0.1)
    
    for epoch in tqdm(range(num_epochs),"Trained"):
        for i, data in enumerate(train_loader):
            inputs,label=data
            inputs=inputs.to(device)
            label=label.to(device)
            opt.zero_grad()

            if regularizer=="SpectReg":
                loss = spectral_reg_loss(model, criterion, inputs, label, 0.03, 10)
            elif regularizer=="JacReg":
                loss = jac_reg_loss(model, criterion, inputs, label, 1)
            elif regularizer=="FrobReg":
                loss = frobreg_loss(model, criterion, inputs, label, 0.03)

            loss.backward()
            opt.step()
    
        scheduler.step()
         
    return Evaluate(test_loader,model)

### SpectReg


In [22]:
accuracy_list=[]
for i in range(10):
    print(f"Iteration:{i} seed:{seeds[i]}")
    model=BaseModel(include_dropout=True).to(device)
    train_loader=get_subset_dataloader(data_train,seeds[i],num_samples_per_class,batch_size)
    accuracy=train(model,train_loader,test_loader,num_samples_per_class,"SpectReg")
    accuracy_list.append(accuracy)
    print("Accuracy:",accuracy)

Iteration:0 seed:10


Subset created with total lenght: 2000


Trained: 100%|██████████| 250/250 [04:28<00:00,  1.08s/it]


Accuracy: 96.37
Iteration:1 seed:20
Subset created with total lenght: 2000


Trained: 100%|██████████| 250/250 [04:31<00:00,  1.09s/it]


...

Accuracy: 96.4
Iteration:8 seed:90
Subset created with total lenght: 2000


Trained: 100%|██████████| 250/250 [03:40<00:00,  1.14it/s]


Accuracy: 96.74
Iteration:9 seed:100
Subset created with total lenght: 2000


Trained: 100%|██████████| 250/250 [03:47<00:00,  1.10it/s]


Accuracy: 96.54


In [23]:
print("Accuracy List",accuracy_list)
print("Mean Accuracy:",np.mean(accuracy_list))
print("SD:",round(np.std(accuracy_list),2))

Accuracy List [96.37, 96.39, 96.72, 96.74, 96.13, 96.32, 96.52, 96.4, 96.74, 96.54]
Mean Accuracy: 96.487
SD: 0.19


### JacReg


In [24]:
accuracy_list=[]
for i in range(10):
    print(f"Iteration:{i} seed:{seeds[i]}")
    model=BaseModel(include_dropout=True).to(device)
    train_loader=get_subset_dataloader(data_train,seeds[i],num_samples_per_class,batch_size)
    accuracy=train(model,train_loader,test_loader,num_samples_per_class,"JacReg")
    accuracy_list.append(accuracy)
    print("Accuracy:",accuracy)

Iteration:0 seed:10
Subset created with total lenght: 2000


Trained: 100%|██████████| 250/250 [03:55<00:00,  1.06it/s]


Accuracy: 96.09
Iteration:1 seed:20
Subset created with total lenght: 2000


Trained: 100%|██████████| 250/250 [03:51<00:00,  1.08it/s]


Accuracy: 96.46
Iteration:2 seed:30
Subset created with total lenght: 2000


...

Accuracy: 96.84
Iteration:8 seed:90
Subset created with total lenght: 2000


Trained: 100%|██████████| 250/250 [03:53<00:00,  1.07it/s]


Accuracy: 96.78
Iteration:9 seed:100
Subset created with total lenght: 2000


Trained: 100%|██████████| 250/250 [03:42<00:00,  1.12it/s]


Accuracy: 96.39


In [25]:
print(accuracy_list)
print(np.mean(accuracy_list))
print(np.std(accuracy_list))

[96.09, 96.46, 96.65, 96.72, 96.46, 96.61, 96.81, 96.84, 96.78, 96.39]
96.581
0.22156037551872923


### FrobReg


In [26]:
accuracy_list=[]
for i in range(10):
    print(f"Iteration:{i} seed:{seeds[i]}")
    model=BaseModel(include_dropout=True).to(device)
    train_loader=get_subset_dataloader(data_train,seeds[i],num_samples_per_class,batch_size)
    accuracy=train(model,train_loader,test_loader,num_samples_per_class,"FrobReg")
    accuracy_list.append(accuracy)
    print("Accuracy:",accuracy)

Iteration:0 seed:10
Subset created with total lenght: 2000


Trained: 100%|██████████| 250/250 [03:58<00:00,  1.05it/s]


Accuracy: 96.91
Iteration:1 seed:20
Subset created with total lenght: 2000


Trained: 100%|██████████| 250/250 [03:50<00:00,  1.09it/s]


Accuracy: 96.46
Iteration:2 seed:30
Subset created with total lenght: 2000


...

Accuracy: 96.91
Iteration:8 seed:90
Subset created with total lenght: 2000


Trained: 100%|██████████| 250/250 [03:48<00:00,  1.10it/s]


Accuracy: 96.75
Iteration:9 seed:100
Subset created with total lenght: 2000


Trained: 100%|██████████| 250/250 [03:50<00:00,  1.08it/s]


Accuracy: 96.29


In [27]:
print(accuracy_list)
print(np.mean(accuracy_list))
print(np.std(accuracy_list))

[96.91, 96.46, 96.71, 96.69, 96.38, 96.76, 96.34, 96.91, 96.75, 96.29]
96.61999999999999
0.2209524835796136
